## Security analysis: constraints overview with pypowsybl-jupyter
- This notebook demonstrates how to **highlight weaknesses** on the network with the help of the **security analysis** and the **network explorer widget**.
- It launches a **security analysis** and compiles the **N-1 limit violations** per branch / voltage level under constraint
- We will visualize differences by providing the widget a **custom styling** to apply on branches and voltage level nodes.

#### 1. Load the network
We load a french network snapshot, at a quiet hour.

In [ ]:
import pypowsybl.network as pn
network = pn.create_eurostag_tutorial_example1_network()

#### 2. Launch the security analysis
- We first define the contingencies to study: all the 400kV lines
- We then launch the security analysis with those contingencies

In [ ]:
voltage_levels = network.get_voltage_levels(attributes=['nominal_v'])
vl_400kV_ids = voltage_levels[voltage_levels['nominal_v'] > 370].index
lines = network.get_lines(attributes=['voltage_level1_id', 'voltage_level2_id'])
lines_400kV = lines[
    lines['voltage_level1_id'].isin(vl_400kV_ids) &
    lines['voltage_level2_id'].isin(vl_400kV_ids)
]
lines_400kV

In [ ]:
import pypowsybl.security as ps
security_analysis = ps.create_analysis()
security_analysis.add_single_element_contingencies(lines_400kV.index.to_list());
result = security_analysis.run_ac(network)
result.limit_violations

#### 3. Building the dataframe of lines under constraint in N-1 state 

- Filtering out the N limit violations

In [ ]:
lv_post_contingency = result.limit_violations.copy().reset_index()
lv_post_contingency = lv_post_contingency[lv_post_contingency['contingency_id'] != '']
lv_post_contingency

- Aggregating the results per `subject_id` (line id)

In [ ]:
violations_by_equipment = (
    lv_post_contingency
    .groupby(['subject_id', 'limit_type'])['contingency_id']
    .apply(list)
    .reset_index()
)
violations_by_equipment

- keeping only limit violations of current type

In [ ]:
violations_line = violations_by_equipment[violations_by_equipment['limit_type']=='CURRENT']
violations_line

#### 4. Construct the custom style profile
We highlight the lines under constraint with a very thick stroke width and a pink color

- first taking the default style profile

In [ ]:
default_pf = network.get_default_nad_profile()

- then overriding the edges (branches) styles to highlight the lines under constraint for some of the studied contingencies

In [ ]:
import pandas as pd

edges_styles_df = pd.DataFrame({
    'id': violations_line['subject_id'],
    'edge1': 'pink',
    'edge2': 'pink',
    'width1': '50px',
    'width2': '50px'
})
edges_styles_df.set_index('id', inplace=True)

- adding the contingency ids on top of those lines

In [ ]:
labels_df = default_pf.branch_labels
labels_df['middle'] = ''
contingency_series = violations_line['contingency_id'].apply(lambda x: ','.join(x))
contingency_series.index = violations_line['subject_id']
labels_df.loc[violations_line['subject_id'], 'middle'] = contingency_series

- remove unneeded voltage level description

In [ ]:
vl_descriptions_df = default_pf.vl_descriptions[default_pf.vl_descriptions['type'] != 'FOOTER']

- creating the custom NAD style profile

In [ ]:
diagram_profile=pn.NadProfile(branch_labels=labels_df, vl_descriptions=vl_descriptions_df, bus_descriptions=default_pf.bus_descriptions,
                              bus_node_styles=default_pf.bus_node_styles, edge_styles=edges_styles_df)

#### 5. Display the results!

In [ ]:
from pypowsybl_jupyter import network_explorer

network_explorer(network, depth=3, nad_profile=diagram_profile)

#### 6. Repeating the same logic for high/low voltage limit violations

- Building the dataframe of high/low voltage limit violations by voltage level

In [ ]:
violations_vl = violations_by_equipment[violations_by_equipment['limit_type']!='CURRENT']
violations_vl = violations_vl.set_index(['subject_id'])
violations_vl

- Aggregating the contingency array

In [ ]:
violations_vl['contingencies'] = violations_vl['contingency_id'].apply(lambda x: ', '.join(x))
violations_vl

- Constructing the dataframe for bus styles

In [ ]:
color_mapping = {
    'HIGH_VOLTAGE': 'orange',
    'LOW_VOLTAGE': 'purple',
}

bus_vl_id = network.get_buses(attributes=['voltage_level_id'])
bus_vl_id['descr'] = bus_vl_id['voltage_level_id'].map(violations_vl['contingencies'])
bus_vl_id['edge-width'] = '100px'
bus_styles_df = bus_vl_id[bus_vl_id['descr'].notnull()][['edge-width']]
bus_styles_df['limit_type'] = bus_vl_id['voltage_level_id'].map(violations_vl['limit_type'])
bus_styles_df['edge'] = bus_styles_df['limit_type'].map(color_mapping)
bus_styles_df['fill'] = bus_styles_df['edge']
del bus_styles_df['limit_type']

- Constructing another custom NAD style profile

In [ ]:
diagram_profile=pn.NadProfile(branch_labels=labels_df, vl_descriptions=vl_descriptions_df, bus_descriptions=pd.DataFrame(),
                              bus_node_styles=bus_styles_df, edge_styles=edges_styles_df)

- Display the results

In [ ]:
network_explorer(network, depth=3, nad_profile=diagram_profile)